# Этап 1. Получение ссылок на книги автора

In [110]:
import re
import pandas as pd
import requests
import json
from multiprocessing import Pool, Lock, Value
from bs4 import BeautifulSoup
from selenium import webdriver
import html
import random
from time import sleep
from operator import setitem
from functools import partial

from collections import defaultdict

from tqdm.notebook import tqdm
from tqdm.contrib.concurrent import thread_map

from itertools import chain

In [2]:
from lxml import etree, html as lhtml

In [125]:
authors_id = []
with open("authors.txt") as file:
    for line in file:
        authors_id.append(line)
authors_id = [int(re.sub('\n', '', i)) for i in authors_id]
authors_id

[15753,
 19769,
 15844,
 19768,
 20948,
 43102,
 19073,
 19071,
 20416,
 17675,
 19076,
 20700,
 28169,
 19805,
 20010,
 16471,
 17717,
 19806,
 43162,
 20935,
 16340,
 20282,
 19075,
 15764,
 16297,
 20591,
 26822,
 19072,
 19725,
 20542,
 17896,
 19070,
 20487,
 19727,
 20540]

In [126]:
def get_page(url, n_attempts = 5, t_sleep = 1, **kwargs):
    for _ in range(n_attempts):
        page = requests.get(url, **kwargs)          
        if page.ok:
            return page
        sleep(t_sleep)
    return page

In [127]:
books_author = defaultdict(list)
respub = 'https://www.respublica.ru'
for a_id in tqdm(authors_id):
    page = respub + '/authors/{}?page='.format(a_id)
    for i in range(1, 11):
        try:
            page_i = get_page(page + str(i))
        except Exception:
            print('Exception')
            print(page + str(i) + '\n\n')
            continue

        if not page_i.ok:
            print(page_i.status_code)
            print(page + str(i) + '\n\n')
            continue

        tree = lhtml.fromstring(page_i.text)
        urls = tree.xpath('//a[@class="rd-listing-product-item__image-wrapper"]/attribute::href')
        
        if not urls:
            break
        books_author[a_id].extend(urls)


# 2 этап

In [128]:
doc = set(chain.from_iterable(books_author.values()))

In [130]:
def check(val, pref = '', ind = 0):
    if val:
        return pref + val[ind]
    else:
        return ''
    
def parser(url):
    
    ID = Name = author = preview = img = description = price = old_price = buy = categor = \
    reviewCount = ratingCount = ratingValue = None
    
    try:
        page = get_page(respub + url)
    except Exception:
        print('Exception')
        print(respub + url + '\n\n')
        return {}
    
    if not page.ok:
        print(page.status_code)
        print(respub + url + '\n\n')
        return {}
    
    tree = lhtml.fromstring(page.text)
    
    URL = respub + url
    
    categor = ';'.join(tree.xpath('//span[@class="rd-page-breadcrumbs-item"]//text()'))
    
    block = tree.xpath('//div[@class="rd-page-product__col-left"]')
    if block:
        block = block[0]
        ID = check(block.xpath('.//div[@class="rd-page-product__article"]//span//text()'))
        Name = check(block.xpath('//h1[@class="rd-page-product__title"]//text()'))
        author = '; '.join(block.xpath('//a[@itemprop="brand"]//text()'))
        img = check(block.xpath('//div[@class="rd-page-product__photo"]//img/attribute::data-zoom-image'), respub)
        preview = check(block.xpath('//a[@class="download-pdf"]/attribute::href'), respub)
        
    block = tree.xpath('//div[@class="rd-page-product__col-right"]')
    if block:
        block = block[0]

        path = block.xpath('//span[@itemprop="aggregateRating"]')
        if path:
            path = path[0]
            reviewCount = check(path.xpath('//meta[@itemprop="reviewCount"]/attribute::content'))
            ratingCount = check(path.xpath('//meta[@itemprop="ratingCount"]/attribute::content'))
            ratingValue = check(path.xpath('//meta[@itemprop="ratingValue"]/attribute::content'))

        buy = check(block.xpath('//span[@class="rd-page-product__buy-text"]//text()'))
        buy = len(re.findall('Купить', buy)) > 0

        price = check(block.xpath('//div[@class="rd-page-product__price"]//text()'))
        price = re.sub(' ','',price)

        old_price = check(block.xpath('//span[@class="prev"]//text()'))
        old_price = re.sub(' ','', old_price)
        
    block = tree.xpath('//div[@class="rd-page-product__row rd-page-product__desc"]')
    if block:
        block = block[0]
        description = check(block.xpath('//div[@class="rd-page-product__desc-body"]//text()'))

        keys = block.xpath('//div[@class="rd-page-product__desc-params"]//*[@itemprop="name"]//text()')
        names = block.xpath('//div[@class="rd-page-product__desc-params"]//*[@itemprop="value"]//text()')
        
    answer = {
        "ID": ID,
        "URL": URL,
        "Название": Name,
        "Автор": author,
        "Превью": preview,
        "Изображение": img,
        "Описание":  description,
        "Цена": price,
        "Цена (старая)": old_price,
        "В наличии": buy,
        "Категория": categor,
        "Число отзывов": reviewCount,
        "Число оценок": ratingCount,
        "Оценка": ratingValue
    }
    
    answer.update(zip(keys, names))
    return answer

In [132]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(uid):
    res = parser(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

In [134]:
with Pool(processes=10) as pool:
    result = pool.map(func_wrapper, doc)
pool.join()

2450 objects are processed...

In [136]:
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)
df.head(4)

,ID,URL,Название,Автор,Превью,Изображение,Описание,Цена,Цена (старая),В наличии,...,Материал,"Размер, см",Тип,Рисунок,Герои,Иллюстрации,Назначение,Пол,Тематика,Эпоха
1443,000033,https://www.respublica.ru/knigi/hudozhestvenna...,Много шума из ничего,Уильям Шекспир,,https://www.respublica.ru/uploads/00/00/00/cq/...,В настоящем издании под одной обложкой объедин...,100,140,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,000056,https://www.respublica.ru/knigi/hudozhestvenna...,Двенадцатая ночь,Уильям Шекспир,,https://www.respublica.ru/uploads/01/00/00/24/...,Перу великого английского драматурга Уильяма Ш...,100,140,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,021678,https://www.respublica.ru/knigi/hudozhestvenna...,N or M?,Agatha Christie,,https://www.respublica.ru/uploads/00/00/00/68/...,Tommy & Tuppence are hired to track down warti...,610,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,021679,https://www.respublica.ru/knigi/hudozhestvenna...,The Secret Adversary,Agatha Christie,,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie's first Tommy and Tuppence mys...,610,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df.iloc[2]

ID                                                                   021678
URL                       https://www.respublica.ru/knigi/hudozhestvenna...
Название                                                            N or M?
Автор                                                       Agatha Christie
Превью                                                                     
Изображение               https://www.respublica.ru/uploads/00/00/00/68/...
Описание                  Tommy & Tuppence are hired to track down warti...
Цена                                                                    610
Цена (старая)                                                              
В наличии                                                             False
Категория                  Книги; Художественная литература; Книги на ин...
Число отзывов                                                          None
Число оценок                                                           None
Оценка      

In [138]:
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)